# Gradient Descent
___

Gradient descent underpins a great deal of the world of deep learning.

Some further links of interest:

* [An overview of gradient descent optimization algorithms](ruder.io/optimizing-gradient-descent/)

## The fundaments

For a given cost function, $J(w)$:

$Repeat\, \{ \\
w := w - \alpha\frac{dJ(w, b)}{dw} \\
b := b - \alpha\frac{dJ(w, b)}{db} \\
\}$

In the context of code, we might abbreviate $\frac{dJ(w, b)}{dw}$ as `dw` for brevity.

### Forward propagation
Consider the case where we have two hidden layers, each with an activation function, with some input vector $X$ of dimension $(n_x, m)$. (This corresponds to a feature vector with $n_x$ features and $m$ samples.

Hence, we define our forward propagation as:

$Z^{[1]} = W^{[1]}X + b^{[1]}$

$A^{[1]} = g^{[1]}(Z^{[1]})$

$Z^{[2]} = W^{[2]}A^{[1]} + b^{[2]}$

$A^{[2]} = g^{[2]}(Z^{[2]}) = \sigma(Z^{[2]})$

### Backward propagation
So now, we've arrived at some collection of outputs which give us an estimation of our classes, $Y$. Working backwards:

$dZ^{[2]} = A^{[2]} - Y$

We recalculate our weights and biases:

$dW^{[2]} = \frac{1}{m}dZ^{[2]}A^{[2]T}$

$dB^{[2]} = \frac{1}{m}{np.sum}(dZ^{[2]}, axis=1, keepdims=True)$

And then step back over the previous layers:

$dZ^{[1]} = W^{[2]T}dZ^{[2]}*g^{[1]'}(Z^{[1]})$

Again, recalculating our weights and biases:

$dW^{[1]} = \frac{1}{m}dZ^{[1]}X^{[1]T}$

$dB^{[1]} = \frac{1}{m}{np.sum}(dZ^{[1]}, axis=1, keepdims=True)$


## Derivatives of activation functions



### ReLU and Leaky ReLU

Here defined as: $g(z) = {max}(0, z)$, we can calculate the derivative $g'(z) in a straightforward manner:

$g'(x) = \begin{cases}
0\ if\ z<0\\
1\ if\ z\ge0\\
\end{cases}$

Or, in the case of leaky ReLU, where $g(z) = {max}(0.01z, z)$,

$g'(x) = \begin{cases}
0.01\ if\ z<0\\
1\ if\ z\ge0\\
\end{cases}$

### Sigmoid
Where $g(z) = \frac{1}{(1+e^{-z})}$,

$g'(z) = \frac{1}{(1+e^{-z})}(1-\frac{1}{(1+e^{-z})})$

or, put more simply:

$g'(z) = g(z)(1-g(z))$

### Tanh
Where $g(z) = \tanh(z)$,

$g'(z) = 1 - \tanh^2(z) = 1 - (g(z))^2$

## Stochastic vs Batch vs Mini-Batch Gradient Descent

Stochastic gradient descent runs gradient descent for each training sample. This is relatively ineffecient and may be too eager too eager to adjust model performance, and moreover, is too slow: We want to take advantage of vectorisation for our training.

With batch gradient descent, we run the whole dataset through the model and adjust the parameters.

In practise, if we have, say, 5,000,000 samples in our training sets, we might not like to wait until we've run all 5,000,000 samples to run gradient descent.

Instead, we split our training set up into "mini-batches", and run one round of gradient descent for each "mini-batch".

We call this "mini-batch gradient descent", in contrast to "batch gradient descent" (which refers to using the whole dataset.

[Reference](https://machinelearningmastery.com/gentle-introduction-mini-batch-gradient-descent-configure-batch-size/)

### Differences in observing training
As we're running over a different dataset each time, we might note that the cost function jumps around (but, generally speaking, trends downwards.)

<img src="static/batch_vs_minibatch_training.png">

### Choosing mini-batch size

If $n_{minibatch} = m$: we've just got gradient descent.

If $n_{minibatch} size = 1$: we've just got stochastic gradient descent.

Typical mini-batch sizes are powers of 2 - 32, 64, 128, etc.

## Components of Gradient Descent

### Exponentially weighted averages

For $n$ noisy, but related points, we might like to use the weight of the previous points to determine an average. One such approach is [exponentially weighted averages](https://en.wikipedia.org/wiki/Moving_average#Exponential_moving_average), where

$v_t = \beta v_{t-1} + (1-\beta) \theta_t$

We can think of $v_t$ as approximating $\frac{1}{1-\beta}$ days temperature, i.e. when $\beta=0.9$, we are approximating 10 day's temperatures.

#### Bias correction

We don't want to start with 0 in our values - this may skew our first few results. So to correct for this initial bias, we might just take the first value as our initial value, or perhaps sample a few values.

[A discussion is here.](https://blog.fugue88.ws/archives/2017-01/The-correct-way-to-start-an-Exponential-Moving-Average-EMA)

## Variations and Flavours of Grad Descent

A good summary of all these flavours are [here](http://ruder.io/optimizing-gradient-descent/index.html#adam).

### RMSprop

This stands for "root mean square" prop.

The key intuition of RMS prop is that we take an exponentially weighted average of the square of the $dW$ and $dB$ terms, and then update the values with our term divided by the square of the average. i.e.

$S_{dW} = \beta S_{dW} + (1-\beta){dW}^2$

$W := W - \alpha \frac{dW}{\sqrt{S_{dW}}}$

The appeal of this is that the weighting and division is carried out for each dimension - we are essentially scaling the gradient change term based on the magnitude of the gradient change.

In practise, we also add a small $\epsilon$ value (e.g. $10^{-8}$) in the denominator to prevent the denominator from going too small and blowing up the value.

### Adam

The key intuition here is that it's a combination of RMSprop and Momentum approaches.

So:

$V_{dW} = \beta_1 V_{dW} + (1-\beta_1)dW$

$S_{dW} = \beta_2 S_{dW} + (1-\beta_2)dW^2$

$W := W - \alpha \frac{V_{dW}}{\sqrt{dW}}$